In [5]:

# fractal_physics_to_quant
# Estimation of fractal dimension (mass–size) and bridge to finance:
# - Log–log fits per material (slope d, CI, R², residuals)
# - Comparison of d vs Euclidean references
# - Volatility vs horizon (synthesis): slope ~ H
# - Power-law tails (Pareto synthesis): slope ~ -α
#
# Requirements: numpy, pandas, matplotlib, seaborn, scipy

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set(context="talk", style="whitegrid", palette="deep")
np.set_printoptions(suppress=True, floatmode="fixed", precision=4)

# --- Configuration ---
# Define the output directory for all generated plots
OUTPUT_DIR = "tail_risk_figures"
# --- End Configuration ---


# ----------------------------
# 1) Experimental data
# ----------------------------
# Tables (mass in g; diameters in mm; 3 measurements per ball)
algae = pd.DataFrame({
    "m_g":   [0.81, 3.16, 4.32, 2.51, 6.31, 9.43, 10.19],
    "D1_mm": [190.1, 294.0, 344.3, 276.6, 377.6, 392.2, 455.3],
    "D2_mm": [188.4, 285.0, 337.5, 290.0, 381.9, 406.7, 421.4],
    "D3_mm": [177.1, 298.8, 336.8, 293.8, 372.8, 411.5, 451.8],
})
paper = pd.DataFrame({
    "m_g":   [9.91, 5.09, 2.33, 1.20, 0.59, 0.28],
    "D1_mm": [521.3, 317.3, 255.0, 174.5, 114.0, 87.2],
    "D2_mm": [493.7, 316.8, 264.1, 186.5, 128.2, 70.9],
    "D3_mm": [491.9, 335.0, 239.8, 179.7, 123.8, 79.6],
})
plasticine = pd.DataFrame({
    "m_g":   [9.65, 15.93, 23.84, 31.82, 41.30],
    "D1_mm": [235.9, 294.3, 312.8, 359.8, 407.6],
    "D2_mm": [255.0, 278.1, 330.0, 369.2, 409.2],
    "D3_mm": [246.0, 292.2, 314.1, 353.6, 391.9],
})

def prepare(df):
    df = df.copy()
    df["Dbar_mm"] = df[["D1_mm", "D2_mm", "D3_mm"]].mean(axis=1)
    df["logD"] = np.log10(df["Dbar_mm"])
    df["logm"] = np.log10(df["m_g"])
    return df

algae = prepare(algae)
paper = prepare(paper)
plasticine = prepare(plasticine)

materials = {
    "Algae": algae,
    "Paper": paper,
    "Plasticine": plasticine
}

# ----------------------------
# 2) OLS log–log fit and utilities
# ----------------------------
def fit_loglog(df):
    x = df["logD"].to_numpy()
    y = df["logm"].to_numpy()
    lr = stats.linregress(x, y)
    slope, intercept = lr.slope, lr.intercept
    r2 = lr.rvalue**2
    stderr = lr.stderr  # SE of the slope
    n = len(x)
    yhat = intercept + slope * x
    resid = y - yhat
    sse = np.sum(resid**2)
    rmse = np.sqrt(sse / (n - 2))
    # 95% CI for the slope
    tcrit = stats.t.ppf(0.975, df=n-2)
    ci_low = slope - tcrit * stderr
    ci_high = slope + tcrit * stderr
    return {
        "slope": slope, "intercept": intercept, "stderr": stderr,
        "ci": (ci_low, ci_high), "r2": r2, "rmse": rmse,
        "x": x, "y": y, "yhat": yhat, "resid": resid, "n": n
    }

def plot_loglog_fit(name, res):
    plt.figure(figsize=(8,6))
    plt.scatter(res["x"], res["y"], s=70, label="Data")
    xgrid = np.linspace(min(res["x"])*0.98, max(res["x"])*1.02, 100)
    ygrid = res["intercept"] + res["slope"] * xgrid
    ci = res["ci"]
    plt.plot(xgrid, ygrid, color="tab:red", lw=2.5,
             label=f"Fit: d ≈ {res['slope']:.2f}  (95%CI [{ci[0]:.2f},{ci[1]:.2f}], R²={res['r2']:.3f})")
    plt.xlabel("log10(Mean Diameter, mm)")
    plt.ylabel("log10(Mass, g)")
    plt.title(f"{name}: Mass–Size Scaling (log–log)")
    plt.legend()
    plt.tight_layout()
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    plt.savefig(f"{OUTPUT_DIR}/{name.lower()}_loglog_fit.png", dpi=200)
    plt.close()

def plot_residuals(name, res):
    plt.figure(figsize=(8,5))
    plt.axhline(0, color="k", lw=1)
    plt.scatter(res["yhat"], res["resid"], s=60)
    plt.xlabel("Fitted value ŷ (log10 m)")
    plt.ylabel("Residual (log10 m)")
    plt.title(f"{name}: Residuals vs. Fitted")
    plt.tight_layout()
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    plt.savefig(f"{OUTPUT_DIR}/{name.lower()}_residuals.png", dpi=200)
    plt.close()

# ----------------------------
# 3) Fits per material
# ----------------------------
fits = {name: fit_loglog(df) for name, df in materials.items()}
for name, res in fits.items():
    plot_loglog_fit(name, res)
    plot_residuals(name, res)

# Comparison of slopes
names = list(fits.keys())
slopes = [fits[n]["slope"] for n in names]
errs =   [fits[n]["stderr"] for n in names]
ns =     [fits[n]["n"] for n in names]
cis = []
for n, s, se in zip(ns, slopes, errs):
    tcrit = stats.t.ppf(0.975, df=n-2)
    cis.append((s - tcrit*se, s + tcrit*se))

lower_err = [s - c[0] for s, c in zip(slopes, cis)]
upper_err = [c[1] - s for s, c in zip(slopes, cis)]

plt.figure(figsize=(9,6))
plt.bar(names, slopes, yerr=[lower_err, upper_err], capsize=6, color=["#4C72B0","#55A868","#C44E52"])
plt.axhline(3.0, ls="--", c="k", lw=1.2, label="Solid reference (d=3)")
plt.axhline(2.0, ls=":", c="gray", lw=1.2, label="Surface reference (d=2)")
plt.ylabel("Estimated fractal dimension d")
plt.title("Comparison of d per material (95% CI)")
plt.legend()
plt.tight_layout()
os.makedirs(OUTPUT_DIR, exist_ok=True)
plt.savefig(f"{OUTPUT_DIR}/dimension_comparison.png", dpi=200)
plt.close()

# ----------------------------
# 4) Finance (scaling analogues)
# ----------------------------
# 4a) Volatility vs horizon (Hurst ~ 0.5) with synthetic Gaussian returns
rng = np.random.default_rng(42)
n = 20000
ret = rng.normal(0.0, 1.0, size=n)  # i.i.d. returns

def std_by_horizon(r, horizons):
    xs, ys = [], []
    for h in horizons:
        m = len(r) // h
        blocks = r[:m*h].reshape(m, h).sum(axis=1)  # h-step sums (cumulative returns)
        xs.append(np.log10(h))
        ys.append(np.log10(np.std(blocks, ddof=1)))
    return np.array(xs), np.array(ys)

horizons = [1,2,4,8,16,32,64]
xv, yv = std_by_horizon(ret, horizons)
lr_vol = stats.linregress(xv, yv)
H_est = lr_vol.slope  # slope ~ H

plt.figure(figsize=(8,6))
plt.scatter(xv, yv, s=70, label="std per horizon")
xgrid = np.linspace(min(xv), max(xv), 100)
plt.plot(xgrid, lr_vol.intercept + lr_vol.slope*xgrid, color="tab:red", lw=2.5,
         label=f"Fit: slope ≈ {H_est:.3f} (expected H ≈ 0.5)")
plt.xlabel("log10(horizon Δ)")
plt.ylabel("log10(std of sums over Δ)")
plt.title("Volatility vs. Horizon (Gaussian synthesis)")
plt.legend()
plt.tight_layout()
os.makedirs(OUTPUT_DIR, exist_ok=True) # Ensure directory exists
plt.savefig(f"{OUTPUT_DIR}/vol_vs_horizon.png", dpi=200)
plt.close()

# 4b) Power-law tails (Pareto): CCDF ~ x^{-α}
alpha_true = 2.5
xm = 1.0
U = rng.random(200000)
X = xm * (1.0 - U)**(-1.0/alpha_true)  # Pareto(α)
X_sorted = np.sort(X)
nX = len(X_sorted)
ccdf = 1.0 - np.arange(1, nX+1) / (nX + 1.0)

# Fit the upper tail (x >= 90th percentile)
x_min = np.quantile(X_sorted, 0.9)
mask = X_sorted >= x_min
x_tail = np.log10(X_sorted[mask])
y_tail = np.log10(ccdf[mask])

lr_tail = stats.linregress(x_tail, y_tail)
alpha_hat = -lr_tail.slope

plt.figure(figsize=(8,6))
plt.scatter(np.log10(X_sorted), np.log10(ccdf), s=6, alpha=0.4, label="CCDF (log–log)")
xgrid = np.linspace(min(x_tail), max(x_tail), 200)
plt.plot(xgrid, lr_tail.intercept + lr_tail.slope*xgrid, color="tab:red", lw=2.0,
         label=f"Tail fit (x≥p90): slope ≈ {lr_tail.slope:.2f} -> alpha_hat ≈ {alpha_hat:.2f}")
plt.xlabel("log10(x)")
plt.ylabel("log10 P(X > x)")
plt.title(f"Power-Law Tails (Pareto α={alpha_true}) – Log-log Estimation")
plt.legend()
plt.tight_layout()
os.makedirs(OUTPUT_DIR, exist_ok=True) # Ensure directory exists
plt.savefig(f"{OUTPUT_DIR}/power_law_tails_ccdf.png", dpi=200)
plt.close()

# ----------------------------
# 5) Console summary
# ----------------------------
print("\n=== Fractal Dimension (mass–size) ===")
for name in names:
    res = fits[name]
    lo, hi = res["ci"]
    print(f"{name:12s} d ≈ {res['slope']:.3f}  (95%CI [{lo:.3f}, {hi:.3f}]), R²={res['r2']:.3f}, RMSE={res['rmse']:.3f}")

print("\n=== Finance: scaling ===")
print(f"Hurst (vol vs horizon, synthetic Gaussian): Ĥ ≈ {H_est:.3f} (expected ≈ 0.5)")
print(f"Pareto tails: true α = {alpha_true:.2f}, α̂ (p90 tail) ≈ {alpha_hat:.2f}")

print(f"\nFigures saved in ./{OUTPUT_DIR}/")



=== Fractal Dimension (mass–size) ===
Algae        d ≈ 3.002  (95%CI [2.605, 3.399]), R²=0.987, RMSE=0.048
Paper        d ≈ 1.985  (95%CI [1.750, 2.220]), R²=0.993, RMSE=0.055
Plasticine   d ≈ 2.971  (95%CI [2.430, 3.511]), R²=0.990, RMSE=0.028

=== Finance: scaling ===
Hurst (vol vs horizon, synthetic Gaussian): Ĥ ≈ 0.510 (expected ≈ 0.5)
Pareto tails: true α = 2.50, α̂ (p90 tail) ≈ 2.53

Figures saved in ./tail_risk_figures/
